In [ ]:
!pip install transformers sentence-transformers -Uqqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, GenerationConfig, AutoModel, BertLMHeadModel
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from sentence_transformers import util
import torch

In [ ]:
import pandas as pd

df = pd.read_csv("/content/machine_learning_dataframe.csv")
df.head()

,Unnamed: 0,parent_id,node_id,memory,Unnamed: 4
0,0,ROOT_1,Machine learning,This book is organized roughly as follows:\nCh...,NaN
1,1,ROOT_1,1. Introduction,"In[29]:\ny_pred = knn.predict(X_test)\nprint(""...",NaN
2,2,1. Introduction,Why Machine Learning?,spam. This would be an example of using an exp...,NaN
3,3,Why Machine Learning?,Problems Machine Learning Can Solve,spam. This would be an example of using an exp...,NaN
4,4,Why Machine Learning?,Knowing Your Task and Knowing Your Data,Segmenting customers into groups with similar ...,NaN


In [ ]:
df_ml = df[df["parent_id"] == "Categorical Variables"]
df_ml

,Unnamed: 0,parent_id,node_id,memory,Unnamed: 4
129,64,Categorical Variables,One-Hot-Encoding (Dummy Variables),"As a starting point, let’s say we want to lear...",NaN
130,65,Categorical Variables,Numbers Can Encode Categoricals,Numbers Can Encode Categoricals\nIn the exampl...,NaN


In [ ]:
list(df_ml['node_id'])

['Approaching a Machine Learning Problem',
 'From Prototype to Production',
 'Testing Production Systems',
 'Building Your Own Estimator',
 'Where to Go from Here',
 'Conclusion']

In [ ]:

question_tokenizer = AutoTokenizer.from_pretrained("tau/spider-nq-question-encoder")
question_model = AutoModel.from_pretrained("tau/spider-nq-question-encoder")

context_tokenizer = AutoTokenizer.from_pretrained("tau/spider-nq-ctx-encoder")
context_model = DPRContextEncoder.from_pretrained("tau/spider-nq-ctx-encoder")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at tau/spider-nq-ctx-encoder were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-360M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-360M")


tokenizer_config.json:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='HuggingFaceTB/SmolLM-360M', vocab_size=49152, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=Tr

In [ ]:
from transformers import  GenerationConfig

In [ ]:
def query_expansion(query:str):


    input_ids = tokenizer(query,  return_tensors = "pt")
    output = model.generate(input_ids=input_ids['input_ids'], generation_config=GenerationConfig(max_new_tokens =  30))
    output = tokenizer.decode(output[0], skip_special_tokens=True) #original

    return output

In [ ]:
def calculate_relevancy(question_embeddings, docs):
    scores = []
    for doc in docs:
        if isinstance(doc, str) and doc: # Check if doc is a non-empty string
            context_input_ids = context_tokenizer(doc,padding='max_length', truncation=True, max_length=512, return_tensors="pt")["input_ids"]
            context_embeddings = context_model(context_input_ids).pooler_output
            score = torch.cosine_similarity(question_embeddings, context_embeddings)
            scores.append(score.item())
        else:
            scores.append(0) # Assign a score of 0 for invalid documents
    return scores

In [ ]:
def rag_generation(query:str):

    probability = 0
    query = query_expansion(query)
    print(query)
    question_input_ids = question_tokenizer(query,padding=True,truncation = True,max_length= 512, return_tensors="pt")["input_ids"]
    question_embeddings = question_model(question_input_ids).pooler_output
    current_node = "ROOT_1"

    while (probability < 0.85):

         data= df[df["parent_id"] == current_node]
         docs = list(data['memory'])
         #docs = [doc for doc in docs if doc not in ['', None]] # Handle potential empty or None documents
         print("num paras", len(docs))
         if docs: # Check if docs is not empty before proceeding
             scores = calculate_relevancy(question_embeddings, docs)
             print(current_node)
             print(scores)
             probability = max(scores)
             max_index = scores.index(max(scores))
             #print(max_index)
             current_node = data.node_id.values[max_index]
         else:
             print("No relevant documents found for the current node.")
             break # Exit the loop if no relevant documents are found

    return data, max_index

In [ ]:
question = "what is Overfitting, and Underfitting in machine learning?"

In [ ]:
query_expansion(question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'what is machine learning?\nMachine learning is a branch of artificial intelligence that enables computers to learn from data without being explicitly programmed. It involves training algorithms to identify patterns and make'

In [ ]:
info, index = rag_generation(question)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


what is Overfitting, and Underfitting in machine learning?

Overfitting is a problem where the model is too complex and fits the training data too closely, resulting in poor generalization to new data. Under
num paras 2
ROOT_1
[0.7040188312530518, 0.6248224377632141]
num paras 7
Machine learning
[0.6700710654258728, 0.6479668617248535, 0.6976355910301208, 0.66367506980896, 0.6452316045761108, 0.6715877652168274, 0.6499938368797302]
num paras 7
4. Representing Data and Engineering Features
[0.6701036691665649, 0, 0, 0, 0.6676673889160156, 0, 0]
num paras 2
Categorical Variables
[0.6698368787765503, 0.6701036691665649]
num paras 0
No relevant documents found for the current node.


In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
input_ids = tokenizer(docs,padding=True,  return_tensors="pt")["input_ids"]
context_embeddings = model(input_ids).pooler_output

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
input_ids = tokenizer(question,padding=True, return_tensors="pt")["input_ids"]
question_embeddings = model(input_ids).pooler_output

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("tau/spider-nq-question-encoder")
model = AutoModel.from_pretrained("tau/spider-nq-question-encoder")

input_ids = tokenizer(question,padding=True,  return_tensors="pt")["input_ids"]
question_embeddings = model(input_ids).pooler_output

Some weights of the model checkpoint at tau/spider-nq-question-encoder were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load model directly
from transformers import AutoTokenizer, DPRContextEncoder

tokenizer = AutoTokenizer.from_pretrained("tau/spider-nq-ctx-encoder")
model = DPRContextEncoder.from_pretrained("tau/spider-nq-ctx-encoder")
input_ids = tokenizer(docs,padding=True,  return_tensors="pt")["input_ids"]
context_embeddings = model(input_ids).pooler_output

Some weights of the model checkpoint at tau/spider-nq-ctx-encoder were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
cosine_scores

tensor([[0.8467],
        [0.6700]], grad_fn=<MmBackward0>)

In [ ]:
for score in cosine_scores:
    print(score.item())

0.8466989994049072
0.6700410842895508


In [ ]:
cosine_scores = util.pytorch_cos_sim(context_embeddings, question_embeddings)
cosine_scores

tensor([[0.8467],
        [0.6700]], grad_fn=<MmBackward0>)